In [15]:
import random

In [1]:
%load_ext ipython_unittest

# Bin Packing Problem with Conflicts

### Algorithm 1: ILS para o PBPC (KLB, ItBL) 

```
S ← ConstroiSoluçãoInicial();
K ← Número de bins em S;
enquanto K ≥ KLB and S é viável, faça: 
  ReduzNúmeroBins(S);
  K ← K − 1;
  ItPERT ← 0;
  enquanto ItPERT ≤ ItMAX and S não é viável, faça: 
    S ← BuscaLocal(S);
    S ← EjectionChains(S);
    se ItBL sucessivas BL e EC sem melhora entao˜
      S ← Perturbação(S);
      ItPERT ← ItPERT + 1;
      ItBL ← 0;
      fim se
    fim enquanto
fim enquanto
```

## Utilidades

In [13]:
def get_lower_limit(bin_total_space, fixtures):
    total_size = 0
    for fixture in fixtures:
        total_size += fixture["value"]
    
    lower_limit = total_size / bin_total_space
    
    if lower_limit > int(lower_limit):
        lower_limit = int(lower_limit) + 1

    return lower_limit

In [2]:
def has_conflict(conflicts, item, bins):
    for item_bin in bins["itens"]:
        if item_bin["color"] in conflicts[str(item["color"])]:
            return True
    return False

In [3]:
def is_solution_viable(solution, BIN_TOTAL_SPACE, conflicts):
    for bins in solution:
        if bins["space_left"] < 0:
            return False
        for item in bins["itens"]:
            if has_conflict(conflicts, item, bins):
                return False
    return True

In [4]:
#TODO - fazer realocate olhando se há espaço vazio no outro bin
def realocate(bin_left, bin_right):
    return True

In [5]:
def swap_bins(bin_left, bin_right, bin_size):
    for item_bin_left in bin_left["itens"]:
        for item_bin_right in bin_right["itens"]:
            # Verifica se o item da esquerda é menor e se o espaço no bin dele é maior
            if(item_bin_right["value"] > item_bin_left["value"]):
                if(item_bin_left["value"] + bin_left["space_left"] > item_bin_right["value"] + bin_right["space_left"]):
                    aux = item_bin_left
                    set_item_bin(bin_right, aux)
                    remove_item_bin(bin_left, item_bin_left)
                    aux = item_bin_right
                    set_item_bin(bin_left, aux)
                    remove_item_bin(bin_right, item_bin_right)
                    return bin_left, bin_right                 
        
    return bin_left, bin_right

In [17]:
def reduce_bins(solution):    
    chosen_bin = random.choice(solution)
    solution.remove(chosen_bin)
    number_bins = len(solution)
    for item in chosen_bin["itens"]:
        index_random = random.randint(0, number_bins - 1)
        set_item_bin(solution[index_random], item)
    return solution

In [7]:
def is_bin_avaliable(bins, item, conflicts):
    return bins["space_left"] >= item["value"] and has_conflict(conflicts, item, bins) == False

In [20]:
def set_item_bin(bin, item):
    bin["itens"].append(item)
    bin["space_left"] = bin["space_left"] - item["value"]
    return bin

In [21]:
def remove_item_bin(bin, item):
    bin["space_left"] = bin["space_left"] + item["value"]
    bin["itens"].remove(item)    
    return bin

## Algoritmo de construção

In [11]:
def first_fit_decreasing(itens, BIN_TOTAL_SPACE, conflicts):
    sortedItens = sorted(itens, key=lambda k: k["value"], reverse=True)
    solution = [] #array of bins
    for item in sortedItens:
        if len(solution) == 0:
            solution.append({"itens":[item], "space_left":BIN_TOTAL_SPACE-item["value"]})
            continue
        inserted = False
        for bins in solution:
            if is_bin_avaliable(bins, item, conflicts):
                bins["itens"].append(item)
                bins["space_left"] -= item["value"]
                inserted = True
                break
                
        if not inserted:
            solution.append({"itens":[item], "space_left":BIN_TOTAL_SPACE-item["value"]})
    return solution

## Busca Local

In [9]:
def local_search(bins, bin_size):
    random.shuffle(bins)
    for i in range(1,len(bins)):
        if (bins[i-1]["space_left"] < 0 or bins[i]["space_left"] < 0):
            bins[i-1], bins[i] = swap_bins(bins[i-1], bins[i], bin_size)
    return bins

# Solução

In [22]:
BIN_TOTAL_SPACE = 10
MAX_PERTUBATION = 5

conflicts = {
    "0":[2],
    "1":[],
    "2":[0]
}

fixtures = [
            {"value":5, "color": 0},
            {"value":4, "color": 0},
            {"value":4, "color": 0},
            {"value":4, "color": 0},
            {"value":3, "color": 0},
            {"value":2, "color": 0},
            {"value":2, "color": 0},
            {"value":2, "color": 0},
            {"value":4, "color": 0}
          ]

solution = first_fit_decreasing(fixtures, BIN_TOTAL_SPACE, conflicts)    
min_bins_count = get_lower_limit(BIN_TOTAL_SPACE, fixtures)
bins_count = len(solution)   


while (bins_count >= min_bins_count and is_solution_viable(solution, BIN_TOTAL_SPACE, conflicts)):
    new_solution = reduce_bins(solution)
    bins_count -= 1
    pertubation_count = 0

    while (pertubation_count <= MAX_PERTUBATION and not is_solution_viable(new_solution, BIN_TOTAL_SPACE, conflicts)):
        new_solution = local_search(new_solution, BIN_TOTAL_SPACE)
        #TODO - Ejection

        #TODO - Pertubação

    #caso não ache uma solução com a busca e a pertubação ele para
    if(is_solution_viable(new_solution, BIN_TOTAL_SPACE, conflicts)): 
        solution = new_solution
    else:
        break

KeyboardInterrupt: 

In [23]:
%%unittest_testcase
def test_first_fit(self):
    BIN_TOTAL_SPACE = 5
    fixture = [
                {"value":5},
                {"value":4},
                {"value":3},
                {"value":1},
                {"value":2}
              ]
    
    solutionExpected = [
                        {"itens":[{"value":5}], "space_left": 0},
                        {"itens":[{"value":4},{"value":1}], "space_left": 0},
                        {"itens":[{"value":3},{"value":2}], "space_left": 0},
                     ]
    
    S = first_fit_decreasing(fixture, BIN_TOTAL_SPACE)
    assert S == solutionExpected
    

Fail

E
ERROR: test_first_fit (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 17, in test_first_fit
TypeError: first_fit_decreasing() missing 1 required positional argument: 'conflicts'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


<unittest.runner.TextTestResult run=1 errors=1 failures=0>

In [24]:
%%unittest_testcase

"""
Test has_conflicts
"""

conflicts = {
            "0":[2],
            "1":[],
            "2":[0]
        }

bins = {"itens":[{"value":3, "color": 0}], "space_left": 0}

def test_bins_conflicting(self):
    item = {"value":2, "color": 2}
    conflicted = hasConflict(conflicts, item, bins)
    assert conflicted == True

def test_bins_conflicting(self):
    item = {"value":2, "color": 1}
    conflicted = hasConflict(conflicts, item, bins)
    assert conflicted == False

Fail

E
ERROR: test_bins_conflicting (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 21, in test_bins_conflicting
NameError: name 'hasConflict' is not defined

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


<unittest.runner.TextTestResult run=1 errors=1 failures=0>

In [26]:
%%unittest_testcase

"""
Test is_solution_viable
"""

BIN_TOTAL_SPACE = 5
    

fixture = { "bins":[
                {"itens":[{"value":5, "color": 0}], "space_left": 0},
                {"itens":[{"value":4, "color": 0}, {"value":1, "color": 2}], "space_left": 0},
                {"itens":[{"value":3, "color": 0},{"value":2, "color": 1}], "space_left": 0},
            ],
            "conflicts":{
                "0":[],
                "1":[2],
                "2":[1]
            }
          }

def test_solution_viability_with_conflicts(self):
    conflicts = {
        "0":[2],
        "1":[],
        "2":[0]
    }
    
    response = is_solution_viable(fixture["bins"], BIN_TOTAL_SPACE, conflicts)
    assert response == False

def test_solution_viability_with_bins_overloaded(self):
    bins = [
             {"itens":[{"value":5, "color": 0}], "space_left": 0},
             {"itens":[{"value":4, "color": 0}, {"value":1, "color": 2}], "space_left": 0},
             {"itens":[{"value":5, "color": 0},{"value":2, "color": 1}], "space_left": -2},
           ]

    response = is_solution_viable(bins, BIN_TOTAL_SPACE, fixture["conflicts"])
    assert response == False

def test_solution_viable(self):
    response = is_solution_viable(fixture["bins"], BIN_TOTAL_SPACE, fixture["conflicts"])
    assert response == True

Success

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## Ejection Chains